In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import pandas as pd
# df=pd.read_csv("C:\\Users\\ankur\\Reviews.csv\\Reviews.csv")

In [ ]:
# df_main= df[['Summary', 'Text']]

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(text):
    if isinstance(text,str):
        text = BeautifulSoup(text, 'html.parser').get_text()
        tokens = word_tokenize(text)
        tokens = [token.lower() for token in tokens]
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens if token.strip() != '']
        processed_text = ' '.join(tokens)
        return processed_text
    else:
        return ""
df_main.loc[:,['Text', 'Summary']] = df_main[['Text', 'Summary']].applymap(preprocess)

In [ ]:
# df_main = df_main.dropna().reset_index(drop=True)
# df_main = df_main.drop_duplicates().reset_index(drop=True)
# df_main.shape

In [ ]:
# df_main = df_main[df_main['Summary']!='']
# df_main = df_main[df_main['Text']!='']
# df_main.shape

In [ ]:
# df_main = df_main[['Summary', 'Text']].astype(str)
# df_main

In [29]:
import pickle

# Open the pickle file in binary mode for reading
with open('/kaggle/input/summary1/df_main.pkl', 'rb') as f:
    # Load the data from the file
    df_main = pickle.load(f)

# Now, 'data' contains the Python object that was saved in the pickle file
df_main

,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,delight says it all,this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price there was a wide ...
...,...,...
394463,will not do without,great for sesame chickenthis is a good if not ...
394464,disappointed,im disappointed with the flavor the chocolate ...
394465,perfect for our maltipoo,these stars are small so you can give 1015 of ...
394466,favorite training and reward treat,these are the best treats for training and rew...


In [30]:
df_subset = df_main[:1000]

In [31]:
data = df_subset.reset_index(drop=True)
data

,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,delight says it all,this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price there was a wide ...
...,...,...
995,not hot,not hot at all like the other low star reviewe...
996,not hot not habanero,i have to admit i was a sucker for the large q...
997,best babka,i never in my life tasted such a good babka it...
998,my dog loves these but,i am so convinced these are human animal crack...


In [32]:
pip install rouge


Note: you may need to restart the kernel to use updated packages.


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from rouge import Rouge

In [34]:
# Tokenize data
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'})
tokenizer.pad_token = '<PAD>'  # Set the padding token

# Custom dataset
class ReviewDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = self.data.loc[idx, 'Text']
        summary = f"<BOS> {self.data.loc[idx, 'Summary']} <EOS>"

        review_encoding = self.tokenizer(review, truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        summary_encoding = self.tokenizer(summary, truncation=True, max_length=128, padding='max_length', return_tensors='pt')

        return {
            'input_ids': review_encoding['input_ids'].squeeze(),
            'attention_mask': review_encoding['attention_mask'].squeeze(),
            'labels': summary_encoding['input_ids'].squeeze()
        }

In [35]:
# Split data
def split_data(data, test_size=0.25, random_state=42):
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    return train_data, test_data

train_data, test_data = split_data(data)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Create datasets and dataloaders
train_dataset = ReviewDataset(train_data, tokenizer)
test_dataset = ReviewDataset(test_data, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=data_collator, num_workers=2)

In [36]:

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)  # Resize the embedding layer

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    evaluation_strategy='steps',
    eval_steps=200,
    logging_steps=10,
    save_strategy='steps',
    save_steps=200,
    learning_rate=5e-5,
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
200,3.962100,4.065368
400,3.722800,4.016747


TrainOutput(global_step=470, training_loss=4.686876613535779, metrics={'train_runtime': 322.8462, 'train_samples_per_second': 11.615, 'train_steps_per_second': 1.456, 'total_flos': 979845120000000.0, 'train_loss': 4.686876613535779, 'epoch': 5.0})

In [ ]:
model.save_pretrained("kaggle/working/save_model")


In [ ]:
test_data
test_data.to_csv("kaggle/working/test.csv")

In [37]:
for batch in test_loader:
    print(batch)
    break

{'input_ids': tensor([[   72,   717,  5071,  ..., 50259, 50259, 50259],
        [   72,  2904,  8155,  ..., 50259, 50259, 50259],
        [ 4053,  2861,   262,  ..., 50259, 50259, 50259],
        ...,
        [   72, 18548,  1975,  ..., 50259, 50259, 50259],
        [   69,   415,  3477,  ..., 50259, 50259, 50259],
        [ 2655,  6819,   783,  ..., 50259, 50259, 50259]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[   72,   717,  5071,  ...,  -100,  -100,  -100],
        [   72,  2904,  8155,  ...,  -100,  -100,  -100],
        [ 4053,  2861,   262,  ...,  -100,  -100,  -100],
        ...,
        [   72, 18548,  1975,  ...,  -100,  -100,  -100],
        [   69,   415,  3477,  ...,  -100,  -100,  -100],
        [ 2655,  6819,   783,  ...,  -100,  -100,  -100]])}


In [42]:
import torch
from tqdm import tqdm

# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
generated_summaries = []
reference_summaries = []
for index, row in tqdm(data.iterrows(), total=len(data), desc='processing data'):
    if index == 538 :
        continue  # Skip processing this index
    if index== 859:
        continue
    review = row['Text']
    summary = row['Summary']

    # Skip empty summaries
    if summary.strip() == '':
        continue

    # Tokenize review and summary
    review_encoding = tokenizer(review, truncation=True, max_length=512, padding='max_length', return_tensors='pt')
    summary_encoding = tokenizer(summary, truncation=True, max_length=128, padding='max_length', return_tensors='pt')

    input_ids = review_encoding['input_ids'].to(device)
    attention_mask = review_encoding['attention_mask'].to(device)

    # Generate summary
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=128, num_return_sequences=1, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    
    if outputs is not None and len(outputs) > 0 and outputs[0] is not None:
        # Decode generated summary and reference summary
        tokens = []
        for token_id in outputs[0]:
            if token_id is not None:
                decoded_token = tokenizer.decode(token_id.item())
                if decoded_token == tokenizer.pad_token:
                    break
                tokens.append(decoded_token)

        # Join tokens into a single string
        generated_summary = "".join(tokens) if tokens else ''
        reference_summary = summary

        generated_summaries.append(generated_summary)
        reference_summaries.append(reference_summary)
    else:
        # Handle case where outputs is None or empty
        generated_summaries.append('')
        reference_summaries.append('')

# Compute ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)
print(scores)


processing data: 100%|██████████| 1000/1000 [22:40<00:00,  1.36s/it]


{'rouge-1': {'r': 0.49044381915123364, 'p': 0.048237567641080295, 'f': 0.08426655343227403}, 'rouge-2': {'r': 0.1596052345757639, 'p': 0.011530906407555615, 'f': 0.019990795503886345}, 'rouge-l': {'r': 0.4445283729852865, 'p': 0.04315792205103801, 'f': 0.0754225513985582}}


In [45]:
# Accessing ROUGE-1 scores
rouge_1_f1 = scores['rouge-1']['f']
rouge_1_precision = scores['rouge-1']['p']
rouge_1_recall = scores['rouge-1']['r']

In [46]:
rouge_1_f1

0.08426655343227403

In [53]:
# Input text and summary
input_text = "The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound,and feels comfortable to play. However, some users have reported issues with the tuning stability."
input_summary = "Good for beginners but has tuning stability issues."

# Preprocess the input
input_encoding = tokenizer(input_text, truncation=True, max_length=512, padding='max_length', return_tensors='pt')
summary_encoding = tokenizer(input_summary, truncation=True, max_length=128, padding='max_length', return_tensors='pt')

input_ids = input_encoding['input_ids'].to(device)
attention_mask = input_encoding['attention_mask'].to(device)

# Generate summary
outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=128, num_return_sequences=1, early_stopping=True, pad_token_id=tokenizer.eos_token_id)

# Decode generated summary
if outputs[0] is not None:
    tokens = []
    for token_id in outputs[0]:
        if token_id is not None:
            decoded_token = tokenizer.decode(token_id.item())
            if decoded_token == tokenizer.pad_token:
                break
            tokens.append(decoded_token)
    generated_summary = "".join(tokens)
else:
    generated_summary = ""

# Compute ROUGE scores
rouge_scores = rouge.get_scores(generated_summary, input_summary)
print("ROUGE Scores:", rouge_scores)
# # Decode generated summary
# generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Compute ROUGE scores
# rouge_scores = rouge.get_scores(generated_summary, input_summary)
# print("ROUGE Scores:", rouge_scores)


ROUGE Scores: [{'rouge-1': {'r': 0.75, 'p': 0.18181818181818182, 'f': 0.29268292368828075}, 'rouge-2': {'r': 0.2857142857142857, 'p': 0.058823529411764705, 'f': 0.09756097277810835}, 'rouge-l': {'r': 0.625, 'p': 0.15151515151515152, 'f': 0.24390243588340277}}]


In [54]:
generated_summary

'The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound,and feels comfortable to play. However, some users have reported issues with the tuning stability.'